In [1]:
# stdlib imports --------------------------------------------------- #
import argparse
import dataclasses
import json
import functools
import logging
import pathlib
import time
import types
import typing
import uuid
from typing import Any, Literal

# 3rd-party imports necessary for processing ----------------------- #
import numpy as np
import numpy.typing as npt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import pynwb
import upath
import zarr
import utils

In [2]:
session_id = '626791_2022-08-15'

session = utils.get_nwb(session_id, raise_on_missing=True, raise_on_bad_file=True) 

In [3]:
session 

Data type,float64
Shape,"(274256,)"
Array size,2.09 MiB
Chunk shape,None
Compression,None
Compression opts,None
Compression ratio,1.0
Data type,float64
Shape,"(274259,)"
Array size,2.09 MiB
Chunk shape,None


In [8]:
session.epochs[:].query('script_name == "DynamicRouting1"')[['start_time', 'stop_time']]

,start_time,stop_time
id,,
0,80.0712,3698.306


In [10]:
session.trials.start_time[0], session.trials.stop_time[-1]

(np.float64(85.7625), np.float64(3682.84621))

In [9]:
session.units[:].obs_intervals

id
0       [[10.528832310745369, 4572.419075979189]]
1       [[10.528832310745369, 4572.419075979189]]
2       [[10.528832310745369, 4572.419075979189]]
3       [[10.528832310745369, 4572.419075979189]]
4       [[10.528832310745369, 4572.419075979189]]
                          ...                    
2929    [[10.532128779326872, 4572.417483664491]]
2930    [[10.532128779326872, 4572.417483664491]]
2931    [[10.532128779326872, 4572.417483664491]]
2932    [[10.532128779326872, 4572.417483664491]]
2933    [[10.532128779326872, 4572.417483664491]]
Name: obs_intervals, Length: 2934, dtype: object

In [10]:
# facial features 
session.processing['behavior']['lp_side_camera'][:]

,ear_base_l_error,ear_base_l_likelihood,ear_base_l_temporal_norm,ear_base_l_x,ear_base_l_y,ear_tip_l_error,ear_tip_l_likelihood,ear_tip_l_temporal_norm,ear_tip_l_x,ear_tip_l_y,...,whisker_pad_l_side_likelihood,whisker_pad_l_side_temporal_norm,whisker_pad_l_side_x,whisker_pad_l_side_y,whisker_pad_l_top_error,whisker_pad_l_top_likelihood,whisker_pad_l_top_temporal_norm,whisker_pad_l_top_x,whisker_pad_l_top_y,timestamps
id,,,,,,,,,,,,,,,,,,,,,
0,79.406761,0.00014947830641176552,NaN,326.39984130859375,226.06680297851562,46.273556,0.0001733447570586577,NaN,328.5680847167969,245.94422912597656,...,2.2040387293031927e-09,NaN,560.9180908203125,20.01579475402832,395.966187,0.9968094825744629,NaN,650.598876953125,485.39508056640625,NaN
1,1.709469,0.9999853372573853,147.263260,198.28065490722656,153.45864868164062,1.451260,0.9999924302101135,166.554764,208.5664520263672,130.44488525390625,...,0.9997093081474304,441.769165,144.07943725585938,166.32200622558594,0.541238,0.9973362684249878,606.265076,139.0847625732422,159.9558563232422,5.06683
2,1.670960,0.9999844431877136,0.136048,198.37628173828125,153.36187744140625,1.423392,0.9999932050704956,0.041109,208.58119201660156,130.40650939941406,...,0.9996731281280518,0.055483,144.1182861328125,166.2823944091797,0.591522,0.9971482157707214,0.089445,139.10531616210938,159.86880493164062,5.08350
3,1.653104,0.9999895095825195,0.111020,198.28030395507812,153.3060760498047,1.406104,0.9999934434890747,0.013086,208.56849670410156,130.40333557128906,...,0.9996938109397888,0.043769,144.0852813720703,166.31114196777344,0.595767,0.9970046281814575,0.029383,139.1224365234375,159.89268493652344,5.10016
4,1.681913,0.9999918937683105,0.087850,198.21377563476562,153.2487030029297,1.450814,0.9999933242797852,0.020279,208.5482177734375,130.4032440185547,...,0.9997517466545105,0.062320,144.0283203125,166.28585815429688,0.652071,0.9976785778999329,0.111405,139.01177978515625,159.87979125976562,5.11683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274247,13.828854,0.9981189966201782,0.418787,197.90304565429688,155.9340362548828,11.539735,0.9999752640724182,0.297119,206.9082489013672,133.1383514404297,...,0.998347818851471,0.291690,145.68186950683594,165.77655029296875,11.368371,0.9988886117935181,0.479139,140.3295440673828,159.81593322753906,4575.79262
274248,18.926159,0.9999884366989136,0.269085,198.13414001464844,155.7961883544922,15.492024,0.9996631741523743,0.833106,206.22271728515625,132.66494750976562,...,0.9980002641677856,0.231360,145.45339965820312,165.81300354003906,15.975260,0.9991626739501953,0.657500,139.73573303222656,160.09823608398438,4575.80928
274249,12.030972,0.9984285831451416,0.272066,197.89785766601562,155.93106079101562,10.050280,0.9997788667678833,0.547152,206.08108520507812,133.19345092773438,...,0.9986873865127563,0.202141,145.26025390625,165.7533721923828,9.988068,0.9993910789489746,0.336471,139.97047424316406,159.857177734375,4575.82595


In [29]:
# pupil 

session.processing['behavior']['eye_tracking'][:].pupil_area.values
session.processing['behavior']['eye_tracking'][:].timestamps.values


array([       nan,    5.0789 ,    5.09557, ..., 4575.87333, 4575.88998,
       4575.90666])

In [20]:
# running

session.processing['behavior']['running_speed'].data[:]

session.processing['behavior']['running_speed'].timestamps[:]


array([  80.04944,   80.06596,   80.08257, ..., 4540.48717, 4540.50372,
       4540.5204 ])

In [15]:
# cross modal dprimes

session.intervals['performance'][:].cross_modal_dprime.values

array([2.9743649 , 1.99122532, 1.37395546, 0.        , 1.77479553,
       2.11251779])

In [34]:
# licks
# upper limit on duration, thresholding at 0.5s

session.processing['behavior']['licks'].data[:]
session.processing['behavior']['licks'].timestamps[:]

array([  27.98856,   28.85258,   36.4437 , ..., 3711.79002, 3711.87005,
       3711.94207])